In [ ]:
#pip install plotly pandas dash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load the dataset
df = pd.read_csv("NioPracticeUserbase.csv")

# User Demographics Visualization
fig_user_demographics = px.bar(
    df,
    x='Gender',
    title='Number of Users by Gender',
    labels={'Gender': 'User Gender'},
    color='Gender',
    category_orders={'Gender': ['Male', 'Female']},
)

# Subscription Overview
fig_subscription_overview = px.pie(
    df,
    names='Subscription Type',
    title='Subscription Type Overview',
)

# Monthly Revenue Trend
fig_revenue_trend = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Overall Revenue Trend', 'Segmented Revenue Trend by Subscription Type'),
    shared_xaxes=True,
    vertical_spacing=0.1,
)

# Overall Revenue Trend
overall_revenue = df.groupby('Join Date')['Monthly Revenue'].sum().reset_index()
fig_revenue_trend.add_trace(
    go.Scatter(x=overall_revenue['Join Date'], y=overall_revenue['Monthly Revenue'], mode='lines', name='Overall Revenue'),
    row=1, col=1,
)
fig_revenue_trend.update_xaxes(title_text='Join Date', row=1, col=1)
fig_revenue_trend.update_yaxes(title_text='Monthly Revenue', row=1, col=1)

# Segmented Revenue Trend by Subscription Type
for subscription_type in df['Subscription Type'].unique():
    subset_df = df[df['Subscription Type'] == subscription_type]
    segmented_revenue = subset_df.groupby('Join Date')['Monthly Revenue'].sum().reset_index()
    fig_revenue_trend.add_trace(
        go.Scatter(x=segmented_revenue['Join Date'], y=segmented_revenue['Monthly Revenue'], mode='lines', name=subscription_type),
        row=2, col=1,
    )

fig_revenue_trend.update_xaxes(title_text='Join Date', row=2, col=1)
fig_revenue_trend.update_yaxes(title_text='Monthly Revenue', row=2, col=1)

# Layout settings
fig_user_demographics.update_layout(
    updatemenus=[
        dict(
            type='dropdown',
            x=1.05,
            y=0.7,
            showactive=False,
            buttons=[
                dict(label='All Countries', method='relayout', args=['xaxis.categoryorder', 'total ascending']),
                *[
                    dict(label=country, method='relayout', args=['xaxis.categoryorder', 'array', [country]])
                    for country in df['Country'].unique()
                ],
            ],
        ),
    ],
)

fig_subscription_overview.update_layout(
    updatemenus=[
        dict(
            type='dropdown',
            x=1.05,
            y=0.7,
            showactive=False,
            buttons=[
                dict(label='All Durations', method='restyle', args=['labels', [df['Subscription Type']]]),
                *[
                    dict(label=duration, method='restyle', args=['labels', [df[df['Plan Duration'] == duration]['Subscription Type']]])
                    for duration in df['Plan Duration'].unique()
                ],
            ],
        ),
    ],
)

# Show the plots
fig_user_demographics.show()
fig_subscription_overview.show()
fig_revenue_trend.show()
